In [1]:
import trade_decision as td
from daily_report import every_day_printer, append_to_report_file
from datetime import datetime

In [ ]:
report = every_day_printer(td.TICKERS, date = datetime.today().strftime("%Y-%m-%d"))
print(report)
append_to_report_file(report)

Today is 2025-05-09
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
report = every_day_printer(td.TICKERS, date = "2025-03-14")
print(report)
append_to_report_file(report)